## Variational Inference in Pyro on CIFAR 10

### Data Pre-Processing

#### Unwrapping CIFAR 10 Python Dataset

CIFAR-10 contains 60K 32x32 images divided into 10 clusters. `test_batch` contains 10K samples and `data_batch_{1...5}` contains 10K samples each.

In [5]:
import pickle
import os
import numpy

def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

cifar_path = "cifar-10-batches-py"
batches = [file for file in sorted(os.listdir(cifar_path)) if "_batch" in file]

data_batches = list()
for i, batch in enumerate(batches):
    batch_path = os.path.join(os.curdir, cifar_path, batches[i])
    data_batches.append(unpickle(batch_path))

    print(batch_path)
    print(data_batches[i].keys())

len(data_batches)

./cifar-10-batches-py/data_batch_1
dict_keys([b'batch_label', b'labels', b'data', b'filenames'])
./cifar-10-batches-py/data_batch_2
dict_keys([b'batch_label', b'labels', b'data', b'filenames'])
./cifar-10-batches-py/data_batch_3
dict_keys([b'batch_label', b'labels', b'data', b'filenames'])
./cifar-10-batches-py/data_batch_4
dict_keys([b'batch_label', b'labels', b'data', b'filenames'])
./cifar-10-batches-py/data_batch_5
dict_keys([b'batch_label', b'labels', b'data', b'filenames'])


5